In [23]:
import json
import string
import random 
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer 
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout

In [25]:
data_file = open('intents2.json').read()
data = json.loads(data_file)
data

{'intents': [{'tag': 'ERP_definition',
   'patterns': ['What is ERP?', 'Define ERP', 'Explain ERP'],
   'responses': ['ERP stands for Enterprise Resource Planning. It integrates all the processes needed to run a company into a single system.']},
  {'tag': 'ERP_benefits',
   'patterns': ['What are the benefits of ERP?',
    'Why use ERP?',
    'Advantages of ERP'],
   'responses': ['ERP systems increase efficiency, improve collaboration, and provide real-time data insights, leading to better decision-making.']},
  {'tag': 'ERP_modules',
   'patterns': ['What are the modules in ERP?',
    'List ERP modules',
    'ERP components'],
   'responses': ['Common ERP modules include finance, human resources, manufacturing, supply chain management, and customer relationship management (CRM).']},
  {'tag': 'ERP_implementation',
   'patterns': ['How to implement ERP?',
    'ERP implementation process',
    'Steps to implement ERP'],
   'responses': ['ERP implementation involves planning, designing,

In [27]:
#4 Extracting data_X(features) and data_Y(Target)

words = [] #For Bow model/ vocabulary for patterns
classes = [] #For Bow  model/ vocabulary for tags
data_X = [] #For storing each pattern
data_y = [] #For storing tag corresponding to each pattern in data_X 
# Iterating over all the intents

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern) # tokenize each pattern 
        words.extend(tokens) #and append tokens to words
        data_X.append(pattern) #appending pattern to data_X
        data_y.append(intent["tag"]) ,# appending the associated tag to each pattern 
    
    # adding the tag to the classes if it's not there already 
    if intent["tag"] not in classes:
        classes.append(intent["tag"])

# initializing lemmatizer to get stem of words
lemmatizer = WordNetLemmatizer()

# lemmatize all the words in the vocab and convert them to lowercase
# if the words don't appear in punctuation
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
# sorting the vocab and classes in alphabetical order and taking the # set to ensure no duplicates occur
words = sorted(set(words))
classes = sorted(set(classes))

In [29]:
# 5 Text to Numbers
training = []
out_empty = [0] * len(classes)
# creating the bag of words model
for idx, doc in enumerate(data_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    # mark the index of class that the current pattern is associated
    # to
    output_row = list(out_empty)
    output_row[classes.index(data_y[idx])] = 1
    # add the one hot encoded BoW and associated classes to training 
    training.append([bow, output_row])
# shuffle the data and convert it to an array
random.shuffle(training)
training = np.array(training, dtype=object)
# split the features and target labels
train_X = np.array(list(training[:, 0]))
train_Y = np.array(list(training[:, 1]))

In [31]:
#6 The Neural Network Model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_X[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_Y[0]), activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model.summary())
model.fit(x=train_X, y=train_Y, epochs=150, verbose=1)

C:\Users\prathmesh.pandhare\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\prathmesh.pandhare\anaconda3\Lib\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 128)            │        10,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 20)             │         1,300 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,052 (78.33 KB)

 Trainable params: 20,052 (78.33 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0542 - loss: 3.0516
Epoch 2/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0549 - loss: 3.0110 
Epoch 3/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1736 - loss: 2.8228 
Epoch 4/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2167 - loss: 2.7240 
Epoch 5/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2069 - loss: 2.6856 
Epoch 6/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2500 - loss: 2.5920 
Epoch 7/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3799 - loss: 2.4586 
Epoch 8/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3465 - loss: 2.4081 
Epoch 9/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4451 - loss: 2.2181 
Epoch 10/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3812 - loss: 2.0954 
Epoch 11/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5333 - loss: 1.9116 
Epoch 12/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5972 -

In [33]:
model.save('chatbot_model.h5')

In [37]:
import tkinter as tk
from tkinter import scrolledtext
from PIL import Image, ImageTk
import json
import numpy as np
from tensorflow.keras.models import load_model
from nltk.stem import WordNetLemmatizer
import nltk
import random

# Load the model and necessary data
model = load_model('chatbot_model.h5')  # Replace with your model path
lemmatizer = WordNetLemmatizer()
data = json.loads(open('intents2.json').read())  # Replace with your intents file path

# Extract words and classes from the JSON data
words = []
classes = []
for intent in data['intents']:
    for pattern in intent['patterns']:
        # Tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # Add to our classes list
    if intent['tag'] not in classes:
        classes.append(intent['tag'])

# Lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ['?', '!', '.', ',']]
words = sorted(list(set(words)))

# Sort classes
classes = sorted(list(set(classes)))

# Function to clean up the user input
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# Function to convert sentence into bag of words
def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print(f"found in bag: {w}")
    return np.array(bag)

# Function to predict the class of the user input
def pred_class(sentence, words):
    bow_array = bow(sentence, words, show_details=False)
    res = model.predict(np.array([bow_array]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

# Function to get the response based on predicted class
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

# Function to handle user input and display chatbot responses
def send_message(user_message):
    print(f"send_message called with: {user_message}")  # Debug print
    if user_message.strip() != "":
        intents = pred_class(user_message, words)
        print(f"Predicted intents: {intents}")  # Debug print
        result = get_response(intents, data)
        print(f"Chatbot response: {result}")  # Debug print
        chat_history.insert(tk.END, f"You: {user_message}\n", 'user')
        chat_history.insert(tk.END, f"ChatBot: {result}\n\n", 'bot')
        chat_history.see(tk.END)
        entry_box.delete(0, tk.END)  # Clear the entry box after sending the message

# Function to handle button clicks
def on_button_click(pattern):
    print(f"Button clicked with pattern: {pattern}")  # Debug print
    send_message(pattern)

# Create the main window
root = tk.Tk()
root.title("LightHouseIndia")
root.configure(bg='#f0f0f0')  # Set background color

# Load and display the logo
logo_image = Image.open("logo.png")  # Replace with your logo path
logo_image = logo_image.resize((100, 100))  # Resize as needed
logo_photo = ImageTk.PhotoImage(logo_image)

logo_label = tk.Label(root, image=logo_photo, bg='#f0f0f0')
logo_label.pack(pady=10)

# Create a frame for the chat history
frame = tk.Frame(root, bg='#f0f0f0')
frame.pack(pady=10)

# Create a scrolled text widget to display the chat history
chat_history = scrolledtext.ScrolledText(frame, width=50, height=20, wrap=tk.WORD, bg='#e6e6e6', fg='#000000', font=('Arial', 12))
chat_history.pack(expand=True, fill="both")
chat_history.tag_configure('user', foreground='#007BFF', font=('Arial', 12, 'bold'))  # User text style
chat_history.tag_configure('bot', foreground='#FF5733', font=('Arial', 12, 'italic'))  # Bot text style

# Create a frame for the buttons
button_frame = tk.Frame(root, bg='#f0f0f0')
button_frame.pack(pady=10)

# Add buttons for each pattern in the intents
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        button = tk.Button(button_frame, text=pattern, command=lambda p=pattern: on_button_click(p), bg='#007BFF', fg='#ffffff', font=('Arial', 12, 'bold'))
        button.pack(pady=2)

# Create a frame for the input box and send button
input_frame = tk.Frame(root, bg='#f0f0f0')
input_frame.pack(pady=10)

# Create an entry box for user input
entry_box = tk.Entry(input_frame, width=40, bg='#ffffff', fg='#000000', font=('Arial', 12))
entry_box.grid(row=0, column=0, padx=5)

# Create a send button to trigger sending the message
send_button = tk.Button(input_frame, text="Send", command=lambda: send_message(entry_box.get()), bg='#007BFF', fg='#ffffff', font=('Arial', 12, 'bold'))
send_button.grid(row=0, column=1, padx=5)

# Function to handle pressing enter in the entry box
def on_enter(event):
    send_message(entry_box.get())

# Bind the enter key to the send_message function
root.bind('<Return>', on_enter)

# Start the main tkinter loop
root.mainloop()


Button clicked with pattern: What is ERP?
send_message called with: What is ERP?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Predicted intents: [{'intent': 'ERP_definition', 'probability': '0.99943966'}]
Chatbot response: ERP stands for Enterprise Resource Planning. It integrates all the processes needed to run a company into a single system.


In [ ]:
import tkinter as tk
from tkinter import scrolledtext
from PIL import Image, ImageTk
import json
import numpy as np
from tensorflow.keras.models import load_model
from nltk.stem import WordNetLemmatizer
import nltk
import random

# Load the model and necessary data
model = load_model('chatbot_model.h5')  # Replace with your model path
lemmatizer = WordNetLemmatizer()
data = json.loads(open('intents2.json').read())  # Replace with your intents file path

# Extract words and classes from the JSON data
words = []
classes = []
for intent in data['intents']:
    for pattern in intent['patterns']:
        # Tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # Add to our classes list
    if intent['tag'] not in classes:
        classes.append(intent['tag'])

# Lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ['?', '!', '.', ',']]
words = sorted(list(set(words)))

# Sort classes
classes = sorted(list(set(classes)))

# Function to clean up the user input
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# Function to convert sentence into bag of words
def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print(f"found in bag: {w}")
    return np.array(bag)

# Function to predict the class of the user input
def pred_class(sentence, words):
    bow_array = bow(sentence, words, show_details=False)
    res = model.predict(np.array([bow_array]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

# Function to get the response based on predicted class
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

# Function to handle user input and display chatbot responses
def send_message(user_message):
    print(f"send_message called with: {user_message}")  # Debug print
    if user_message.strip() != "":
        intents = pred_class(user_message, words)
        print(f"Predicted intents: {intents}")  # Debug print
        result = get_response(intents, data)
        print(f"Chatbot response: {result}")  # Debug print
        chat_history.insert(tk.END, f"You: {user_message}\n", 'user')
        chat_history.insert(tk.END, f"ChatBot: {result}\n\n", 'bot')
        chat_history.see(tk.END)
        entry_box.delete(0, tk.END)  # Clear the entry box after sending the message

# Function to handle button clicks
def on_button_click(pattern):
    print(f"Button clicked with pattern: {pattern}")  # Debug print
    send_message(pattern)

# Create the main window
root = tk.Tk()
root.title("LightHouseIndia")
root.configure(bg='#f0f0f0')  # Set background color

# Load and display the logo
logo_image = Image.open("logo.png")  # Replace with your logo path
logo_image = logo_image.resize((100, 100))  # Resize as needed
logo_photo = ImageTk.PhotoImage(logo_image)

logo_label = tk.Label(root, image=logo_photo, bg='#f0f0f0')
logo_label.pack(pady=10)

# Create a frame for the chat history
frame = tk.Frame(root, bg='#f0f0f0')
frame.pack(pady=10)

# Create a scrolled text widget to display the chat history
chat_history = scrolledtext.ScrolledText(frame, width=50, height=20, wrap=tk.WORD, bg='#e6e6e6', fg='#000000', font=('Arial', 12))
chat_history.pack(expand=True, fill="both")
chat_history.tag_configure('user', foreground='#007BFF', font=('Arial', 12, 'bold'))  # User text style
chat_history.tag_configure('bot', foreground='#FF5733', font=('Arial', 12, 'italic'))  # Bot text style

# Create a frame for the buttons
button_frame = tk.Frame(root, bg='#f0f0f0')
button_frame.pack(pady=10)

# Add buttons for each pattern in the intents
row = 0
column = 0
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        button = tk.Button(button_frame, text=pattern, command=lambda p=pattern: on_button_click(p), bg='#007BFF', fg='#ffffff', font=('Arial', 12, 'bold'))
        button.grid(row=row, column=column, padx=2, pady=2)
        column += 1
        if column > 4:  # Change this number to control how many buttons per row
            column = 0
            row += 1

# Create a frame for the input box and send button
input_frame = tk.Frame(root, bg='#f0f0f0')
input_frame.pack(pady=10)

# Create an entry box for user input
entry_box = tk.Entry(input_frame, width=40, bg='#ffffff', fg='#000000', font=('Arial', 12))
entry_box.grid(row=0, column=0, padx=5)

# Create a send button to trigger sending the message
send_button = tk.Button(input_frame, text="Send", command=lambda: send_message(entry_box.get()), bg='#007BFF', fg='#ffffff', font=('Arial', 12, 'bold'))
send_button.grid(row=0, column=1, padx=5)

# Function to handle pressing enter in the entry box
def on_enter(event):
    send_message(entry_box.get())

# Bind the enter key to the send_message function
root.bind('<Return>', on_enter)

# Start the main tkinter loop
root.mainloop()


Button clicked with pattern: Explain ERP
send_message called with: Explain ERP
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Predicted intents: [{'intent': 'ERP_definition', 'probability': '0.9981007'}]
Chatbot response: ERP stands for Enterprise Resource Planning. It integrates all the processes needed to run a company into a single system.
Button clicked with pattern: Explain ERP
send_message called with: Explain ERP
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Predicted intents: [{'intent': 'ERP_definition', 'probability': '0.9981007'}]
Chatbot response: ERP stands for Enterprise Resource Planning. It integrates all the processes needed to run a company into a single system.
Button clicked with pattern: List ERP modules
send_message called with: List ERP modules
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Predicted intents: [{'intent': 'ERP_modules', 'probability': '0.9999747'}]
Chatbot response: Common ERP modules include finance, human resources, manufacturing, supply chain management, and customer re

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\prathmesh.pandhare\anaconda3\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\prathmesh.pandhare\AppData\Local\Temp\ipykernel_2336\2352450758.py", line 146, in on_enter
    send_message(entry_box.get())
  File "C:\Users\prathmesh.pandhare\AppData\Local\Temp\ipykernel_2336\2352450758.py", line 81, in send_message
    result = get_response(intents, data)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\prathmesh.pandhare\AppData\Local\Temp\ipykernel_2336\2352450758.py", line 67, in get_response
    tag = intents_list[0]['intent']
          ~~~~~~~~~~~~^^^
IndexError: list index out of range


send_message called with: thanks
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Predicted intents: []


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\prathmesh.pandhare\anaconda3\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\prathmesh.pandhare\AppData\Local\Temp\ipykernel_2336\2352450758.py", line 146, in on_enter
    send_message(entry_box.get())
  File "C:\Users\prathmesh.pandhare\AppData\Local\Temp\ipykernel_2336\2352450758.py", line 81, in send_message
    result = get_response(intents, data)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\prathmesh.pandhare\AppData\Local\Temp\ipykernel_2336\2352450758.py", line 67, in get_response
    tag = intents_list[0]['intent']
          ~~~~~~~~~~~~^^^
IndexError: list index out of range


send_message called with: thankss
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Predicted intents: []


...

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\prathmesh.pandhare\anaconda3\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\prathmesh.pandhare\AppData\Local\Temp\ipykernel_2336\2352450758.py", line 146, in on_enter
    send_message(entry_box.get())
  File "C:\Users\prathmesh.pandhare\AppData\Local\Temp\ipykernel_2336\2352450758.py", line 81, in send_message
    result = get_response(intents, data)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\prathmesh.pandhare\AppData\Local\Temp\ipykernel_2336\2352450758.py", line 67, in get_response
    tag = intents_list[0]['intent']
          ~~~~~~~~~~~~^^^
IndexError: list index out of range
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\prathmesh.pandhare\anaconda3\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\prathme

send_message called with: hei
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Predicted intents: []
send_message called with: hei
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\prathmesh.pandhare\anaconda3\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\prathmesh.pandhare\AppData\Local\Temp\ipykernel_2336\2352450758.py", line 141, in <lambda>
    send_button = tk.Button(input_frame, text="Send", command=lambda: send_message(entry_box.get()), bg='#007BFF', fg='#ffffff', font=('Arial', 12, 'bold'))
                                                                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\prathmesh.pandhare\AppData\Local\Temp\ipykernel_2336\2352450758.py", line 81, in send_message
    result = get_response(intents, data)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\prathmesh.pandhare\AppData\Local\Temp\ipykernel_2336\2352450758.py", line 67, in get_response
    tag = intents_list[0]['intent']
          ~~~~~~~~~~~~^^^
IndexError: list index out of range


Predicted intents: []


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\prathmesh.pandhare\anaconda3\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\prathmesh.pandhare\AppData\Local\Temp\ipykernel_2336\2352450758.py", line 141, in <lambda>
    send_button = tk.Button(input_frame, text="Send", command=lambda: send_message(entry_box.get()), bg='#007BFF', fg='#ffffff', font=('Arial', 12, 'bold'))
                                                                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\prathmesh.pandhare\AppData\Local\Temp\ipykernel_2336\2352450758.py", line 81, in send_message
    result = get_response(intents, data)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\prathmesh.pandhare\AppData\Local\Temp\ipykernel_2336\2352450758.py", line 67, in get_response
    tag = intents_list[0]['intent']
          ~~~~~~~~~~~~^^^
IndexError: list index out of range


In [63]:
import tkinter as tk
from tkinter import scrolledtext, ttk
from PIL import Image, ImageTk
import json
import numpy as np
from tensorflow.keras.models import load_model
from nltk.stem import WordNetLemmatizer
import nltk
import random

# Load the model and necessary data
model = load_model('chatbot_model.h5')  # Replace with your model path
lemmatizer = WordNetLemmatizer()
data = json.loads(open('intents2.json').read())  # Replace with your intents file path

# Extract words and classes from the JSON data
words = []
classes = []
for intent in data['intents']:
    for pattern in intent['patterns']:
        # Tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # Add to our classes list
    if intent['tag'] not in classes:
        classes.append(intent['tag'])

# Lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ['?', '!', '.', ',']]
words = sorted(list(set(words)))

# Sort classes
classes = sorted(list(set(classes)))

# Function to clean up the user input
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# Function to convert sentence into bag of words
def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print(f"found in bag: {w}")
    return np.array(bag)

# Function to predict the class of the user input
def pred_class(sentence, words):
    bow_array = bow(sentence, words, show_details=False)
    res = model.predict(np.array([bow_array]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

# Function to get the response based on predicted class
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

# Function to handle user input and display chatbot responses
def send_message(user_message):
    print(f"send_message called with: {user_message}")  # Debug print
    if user_message.strip() != "":
        intents = pred_class(user_message, words)
        print(f"Predicted intents: {intents}")  # Debug print
        result = get_response(intents, data)
        print(f"Chatbot response: {result}")  # Debug print
        chat_history.insert(tk.END, f"You: {user_message}\n", 'user')
        chat_history.insert(tk.END, f"ChatBot: {result}\n\n", 'bot')
        chat_history.see(tk.END)
        entry_box.delete(0, tk.END)  # Clear the entry box after sending the message

# Function to handle dropdown selection
def on_dropdown_select(event):
    selected_question = dropdown.get()
    send_message(selected_question)

# Create the main window
root = tk.Tk()
root.title("LightHouseIndia")
root.configure(bg='#f0f0f0')  # Set background color

# Load and display the logo
logo_image = Image.open("logo.png")  # Replace with your logo path
logo_image = logo_image.resize((100, 100))  # Resize as needed
logo_photo = ImageTk.PhotoImage(logo_image)

logo_label = tk.Label(root, image=logo_photo, bg='#f0f0f0')
logo_label.pack(pady=10)

# Create a frame for the chat history
frame = tk.Frame(root, bg='#f0f0f0')
frame.pack(pady=10)

# Create a scrolled text widget to display the chat history
chat_history = scrolledtext.ScrolledText(frame, width=50, height=20, wrap=tk.WORD, bg='#e6e6e6', fg='#000000', font=('Arial', 12))
chat_history.pack(expand=True, fill="both")
chat_history.tag_configure('user', foreground='#007BFF', font=('Arial', 12, 'bold'))  # User text style
chat_history.tag_configure('bot', foreground='#FF5733', font=('Arial', 12, 'italic'))  # Bot text style

# Create a dropdown menu for the questions
question_list = []
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        question_list.append(pattern)

dropdown = ttk.Combobox(root, values=question_list, font=('Arial', 12))
dropdown.pack(pady=10)
dropdown.bind("<<ComboboxSelected>>", on_dropdown_select)

# Create a frame for the input box and send button
input_frame = tk.Frame(root, bg='#f0f0f0')
input_frame.pack(pady=10)

# Create an entry box for user input
entry_box = tk.Entry(input_frame, width=40, bg='#ffffff', fg='#000000', font=('Arial', 12))
entry_box.grid(row=0, column=0, padx=5)

# Create a send button to trigger sending the message
send_button = tk.Button(input_frame, text="Send", command=lambda: send_message(entry_box.get()), bg='#007BFF', fg='#ffffff', font=('Arial', 12, 'bold'))
send_button.grid(row=0, column=1, padx=5)

# Function to handle pressing enter in the entry box
def on_enter(event):
    send_message(entry_box.get())

# Bind the enter key to the send_message function
root.bind('<Return>', on_enter)

# Start the main tkinter loop
root.mainloop()


In [61]:
import tkinter as tk
from tkinter import scrolledtext
from tkinter import Scrollbar
from PIL import Image, ImageTk
import json
import numpy as np
from tensorflow.keras.models import load_model
from nltk.stem import WordNetLemmatizer
import nltk
import random

# Load the model and necessary data
model = load_model('chatbot_model.h5')  # Replace with your model path
lemmatizer = WordNetLemmatizer()
data = json.loads(open('intents2.json').read())  # Replace with your intents file path

# Extract words and classes from the JSON data
words = []
classes = []
for intent in data['intents']:
    for pattern in intent['patterns']:
        # Tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # Add to our classes list
    if intent['tag'] not in classes:
        classes.append(intent['tag'])

# Lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ['?', '!', '.', ',']]
words = sorted(list(set(words)))

# Sort classes
classes = sorted(list(set(classes)))

# Function to clean up the user input
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# Function to convert sentence into bag of words
def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print(f"found in bag: {w}")
    return np.array(bag)

# Function to predict the class of the user input
def pred_class(sentence, words):
    bow_array = bow(sentence, words, show_details=False)
    res = model.predict(np.array([bow_array]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

# Function to get the response based on predicted class
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

# Function to handle user input and display chatbot responses
def send_message(user_message):
    print(f"send_message called with: {user_message}")  # Debug print
    if user_message.strip() != "":
        intents = pred_class(user_message, words)
        print(f"Predicted intents: {intents}")  # Debug print
        result = get_response(intents, data)
        print(f"Chatbot response: {result}")  # Debug print
        chat_history.insert(tk.END, f"You: {user_message}\n", 'user')
        chat_history.insert(tk.END, f"ChatBot: {result}\n\n", 'bot')
        chat_history.see(tk.END)

# Function to handle button clicks
def on_button_click(pattern):
    print(f"Button clicked with pattern: {pattern}")  # Debug print
    send_message(pattern)

# Create the main window
root = tk.Tk()
root.title("LightHouseIndia")
root.configure(bg='#f0f0f0')  # Set background color

# Load and display the logo
logo_image = Image.open("logo.png")  # Replace with your logo path
logo_image = logo_image.resize((100, 100))  # Resize as needed
logo_photo = ImageTk.PhotoImage(logo_image)

logo_label = tk.Label(root, image=logo_photo, bg='#f0f0f0')
logo_label.pack(pady=10)

# Create a frame for the chat history
frame = tk.Frame(root, bg='#f0f0f0')
frame.pack(pady=10)

# Create a scrolled text widget to display the chat history
chat_history = scrolledtext.ScrolledText(frame, width=50, height=20, wrap=tk.WORD, bg='#e6e6e6', fg='#000000', font=('Arial', 12))
chat_history.pack(expand=True, fill="both")
chat_history.tag_configure('user', foreground='#007BFF', font=('Arial', 12, 'bold'))  # User text style
chat_history.tag_configure('bot', foreground='#FF5733', font=('Arial', 12, 'italic'))  # Bot text style

# Create a horizontal scrollbar for the buttons
button_frame = tk.Frame(root, bg='#f0f0f0')
button_frame.pack(fill='x')

button_canvas = tk.Canvas(button_frame, bg='#f0f0f0', height=60)  # Set a fixed height for the button canvas
button_canvas.pack(side=tk.LEFT, fill='x', expand=True)

button_scrollbar = Scrollbar(button_frame, orient='horizontal', command=button_canvas.xview)
button_scrollbar.pack(side=tk.BOTTOM, fill='x')

button_canvas.configure(xscrollcommand=button_scrollbar.set)

button_inner_frame = tk.Frame(button_canvas, bg='#f0f0f0')
button_canvas.create_window((0, 0), window=button_inner_frame, anchor='nw')

# Add buttons for each pattern in the intents
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        button = tk.Button(button_inner_frame, text=pattern, command=lambda p=pattern: on_button_click(p), bg='#007BFF', fg='#ffffff', font=('Arial', 12, 'bold'))
        button.pack(side=tk.LEFT, padx=2, pady=2)

button_inner_frame.update_idletasks()
button_canvas.config(scrollregion=button_canvas.bbox("all"))

# Start the main tkinter loop
root.mainloop()
